# Neural Networks
- Neural networks are inspired by the way the human brain works and can be used for a wide range of tasks such as image recognition, natural language processing, and even playing games such as chess and Go.
- Neural networks can be used for generative tasks such as creating images, music, and even text.
- Neural networks can be used for reinforcement learning, where an agent learns to take actions in an environment to maximize a reward.
- Neural networks are able to learn and improve over time with the right training data and can reach human-level performance on certain tasks.
- Neural networks can be used to model complex non-linear relationships between inputs and outputs, which makes them very powerful for solving real-world problems.
- Neural networks are able to generalize well on unseen data and make predictions based on patterns learned from the training data.
- Neural networks are widely used in deep learning and have been behind many of the recent breakthroughs in AI and machine learning.
- Neural networks are being used in various fields such as healthcare, finance, transportation, and robotics.
- Neural networks are open-ended, meaning that there is always something new to discover and optimize, which makes it a fascinating field for researchers and practitioners alike.

# Setting Up Necessary Things

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import numpy as np
import pandas as pd
from torch import tensor

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MasterCourse/Week 11 - Building Neural Networks with PyTorch/Lecture 43 - Build Neural Networks

/content/drive/MyDrive/MasterCourse/Week 11 - Building Neural Networks with PyTorch/Lecture 43 - Build Neural Networks


# Get Data

In [4]:
!gdown 1a4TCpVXvqyVhIGfULqCOg4COejFtcZrk
!gdown 1q_WCC0aLa0Xyyz6Um2_y2BDW6rjEfaHE

Downloading...
From: https://drive.google.com/uc?id=1a4TCpVXvqyVhIGfULqCOg4COejFtcZrk
To: /content/adult_train.csv
100% 4.60M/4.60M [00:00<00:00, 32.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q_WCC0aLa0Xyyz6Um2_y2BDW6rjEfaHE
To: /content/adult_test.csv
100% 1.15M/1.15M [00:00<00:00, 10.4MB/s]


In [25]:
train_df = pd.read_csv("adult_train.csv")
test_df = pd.read_csv("adult_test.csv")
train_df.head()

,age,fnlwgt,educational-num,hours-per-week,log-capital-gain,log-capital-loss,private_workclass,postgrad_education,after_HS_education,school_education,...,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_white,gender_male,country_US,high_income
0,71,77253,9,17,0.0,0.0,True,0,1,0,...,False,True,False,False,False,False,True,True,True,False
1,17,329783,6,10,0.0,0.0,True,0,0,1,...,False,False,True,False,False,False,True,False,True,False
2,27,91257,9,40,0.0,0.0,True,0,1,0,...,True,False,False,False,False,False,True,True,False,False
3,43,125577,9,40,0.0,0.0,True,0,1,0,...,False,False,False,False,True,False,False,False,True,False
4,31,137978,13,40,0.0,0.0,True,0,1,0,...,True,False,False,False,False,False,True,True,True,False


In [26]:
train_feature_df = train_df[train_df.columns[:-1]].astype(float)
train_features = tensor(train_feature_df.values, dtype=torch.float)
train_labels = tensor(train_df.high_income, dtype=torch.int)

max_value, _ = train_features.max(dim=0)
train_features /= max_value

In [27]:
test_feature_df = test_df[test_df.columns[:-1]].astype(float)
test_features = tensor(test_feature_df.values, dtype=torch.float)
test_labels = tensor(test_df.high_income, dtype=torch.int)

max_value, _ = test_features.max(dim=0)
test_features /= max_value

# Linear Model to Neural Net

input layer | **24 params**
___
output layer | **1 param**

* tensor([ 0,  1,  2,  3,  4,  5]) => 1-D tensor or flat vector


*   tensor([
           [0],
           [1],
           [2],
           [3],
           [4],
           [5] ]) => 2-D tensor (matrix)

In [28]:
def init_weights():
  torch.manual_seed(42)
  n_params = train_features.shape[1]
  weights = torch.rand(n_params, 1)   #now it is a 2-D matrix. 24 rows and 1 column
  return weights.requires_grad_()

In [29]:
train_labels.shape

torch.Size([39073])

In [30]:
#practise
x = tensor([1,2,3])
x                 # 1-D tensor or flat vector

tensor([1, 2, 3])

* x[:, None] converts 1-D tensor to 2-D matrix
* making it a column vector ((3,1)) instead of a flat 1D tensor ((3,))
* it's added one extra axis
* it's mainly needed for matrix multiplication as 2-D is need for it


In [31]:
x[:, None]

tensor([[1],
        [2],
        [3]])

In [32]:
# i have taken the weights of 2-D dimension and i have to make the labels as 2-D also
train_labels = train_labels[:, None]
test_labels = test_labels[:, None]

In [33]:
train_labels.shape

torch.Size([39073, 1])

In [35]:
def calculate_preds(weights, features):
  mults = features @ weights        # @ also means matrix multiplication. features means train or test features. here features shape is [39073,24] and weights shape [24,1]. the output shape of matrix mul will be [39073,1]
  preds = torch.sigmoid(mults)      #remove linearity
  return preds

In [36]:
def calculate_loss(preds, labels):
  return torch.abs(preds-labels).mean()

In [37]:
def update_weights(weights, lr):
  weights.sub_(weights.grad * lr)
  return

In [38]:
def one_epoch(epoch_no, weights, lr):
  preds = calculate_preds(weights, train_features)
  loss = calculate_loss(preds, train_labels)
  loss.backward()
  with torch.no_grad(): update_weights(weights, lr)
  print(f"Epoch {epoch_no+1} => Loss: {loss}")

In [39]:
def train_model(epochs, lr):
  weights = init_weights()
  for epoch_no in range(epochs):
    one_epoch(epoch_no, weights, lr)
  return weights

In [40]:
weights = train_model(epochs=15, lr=2)

Epoch 1 => Loss: 0.7542339563369751
Epoch 2 => Loss: 0.7539462447166443
Epoch 3 => Loss: 0.7533162236213684
Epoch 4 => Loss: 0.7522116303443909
Epoch 5 => Loss: 0.7503570914268494
Epoch 6 => Loss: 0.7471655607223511
Epoch 7 => Loss: 0.7412629723548889
Epoch 8 => Loss: 0.7289184331893921
Epoch 9 => Loss: 0.6978182196617126
Epoch 10 => Loss: 0.6009696125984192
Epoch 11 => Loss: 0.369525283575058
Epoch 12 => Loss: 0.25718405842781067
Epoch 13 => Loss: 0.24191097915172577
Epoch 14 => Loss: 0.2397891879081726
Epoch 15 => Loss: 0.23940058052539825


In [41]:
def accuracy(weights, features):
  preds = calculate_preds(weights, features)
  results = test_labels == (preds > 0.5).int()
  return results.float().mean()

In [42]:
accuracy(weights, test_features)

tensor(0.7607)

# Shallow Neural Network (added only one hidden layers mainly)

input layer | **24 params**
___
hidden layer | **10 params**
___
output layer | **1 param**

In [44]:
torch.manual_seed(42)

def init_weights(n_hidden=10):
  n_params = train_features.shape[1]                #input layers
  hidden_layer = torch.rand(n_params, n_hidden)     # connecting input layers with hidden layers
  head_layer = torch.rand(n_hidden, 1)              #connecting hidden layers with output layers
  bias = torch.rand(1)[0]                           #torch.rand(1) generates a 1D tensor with one random value.
  return hidden_layer.requires_grad_(), head_layer.requires_grad_(), bias.requires_grad_()

* torch.rand(1) generates a 1D tensor with one random value.
* torch.rand(1)[0] extracts a scalar value (a plain Python number instead of a tensor).
* The output of torch.rand(1) is tensor([0.7854]) (data type is tensor)
* The output of torch.rand(1)[0] is tensor 0.7854 ( data type is tensor)
* By using torch.rand(1).item() we can convert tensor to float



In [45]:
def calculate_preds(weights, features):
  hidden_layer, head_layer, bias = weights
  # print(features.shape, hidden_layer.shape)
  hidden_layer_out = torch.sigmoid(features @ hidden_layer)
  # print(hidden_layer.shape, hidden_layer_out.shape)
  head_layer_out = hidden_layer_out @ head_layer + bias
  # print(hidden_layer_out.shape, head_layer_out.shape)
  return torch.sigmoid(head_layer_out)

In [46]:
def update_weights(weights, lr):
  for layer in weights:
    layer.sub_(layer.grad * lr)        #layers wise update
  return

In [47]:
weights = train_model(epochs=15, lr=2)

Epoch 1 => Loss: 0.7533177137374878
Epoch 2 => Loss: 0.7521417140960693
Epoch 3 => Loss: 0.7489487528800964
Epoch 4 => Loss: 0.7404935956001282
Epoch 5 => Loss: 0.7112254500389099
Epoch 6 => Loss: 0.5498631000518799
Epoch 7 => Loss: 0.2483927607536316
Epoch 8 => Loss: 0.23936550319194794
Epoch 9 => Loss: 0.23927108943462372
Epoch 10 => Loss: 0.23927009105682373
Epoch 11 => Loss: 0.23927009105682373
Epoch 12 => Loss: 0.23927009105682373
Epoch 13 => Loss: 0.23927009105682373
Epoch 14 => Loss: 0.23927009105682373
Epoch 15 => Loss: 0.23927009105682373


In [48]:
%cd /content/drive/MyDrive/MasterCourseML/week-10/updated weights

/content/drive/MyDrive/MasterCourseML/week-10/updated weights


In [49]:
torch.save(weights, "shallow_nnet_weights.pt")

In [50]:
accuracy(weights, test_features)

tensor(0.7607)

# Deep Neural Network   (added multiple hidden layers mainly)

In [56]:
torch.manual_seed(42)

def init_weights():
  n_params = train_features.shape[1]
  hidden_layers = [10, 10, 10]                # 3 hidden layers
  sizes = [n_params] + hidden_layers + [1]    # [24,10,10,10,1]
  #print(sizes)
  count = len(sizes)                          # 5 different layers
  #print(count)

  layers = [torch.rand(sizes[i], sizes[i+1]) for i in range(count-1)]       #connected each layers
  bias   = [torch.rand(1)[0] for i in range(count-1)]                       # taken 5 bias

  for l in layers + bias : l.requires_grad_()                                # applied gradien in all layes and bias
  return layers, bias

In [55]:
# x = init_weights()

[24, 10, 10, 10, 1]
5


In [59]:
def calculate_preds(weights, features):
  layers, bias = weights
  n = len(layers)
  out = features

  for i, layer in enumerate(layers):
    out = out @ layer + bias[i]
    out = torch.sigmoid(out)
  return out

In [60]:
def update_weights(weights, lr):
  layers, bias = weights
  for layer in layers+bias:            #layers wise update
    layer.sub_(layer.grad * lr)
  return

In [61]:
weights = train_model(epochs=15, lr=10)

Epoch 1 => Loss: 0.759865939617157
Epoch 2 => Loss: 0.7597818970680237
Epoch 3 => Loss: 0.7595779299736023
Epoch 4 => Loss: 0.7591457962989807
Epoch 5 => Loss: 0.7581498622894287
Epoch 6 => Loss: 0.7552076578140259
Epoch 7 => Loss: 0.7399405241012573
Epoch 8 => Loss: 0.45806047320365906
Epoch 9 => Loss: 0.23927009105682373
Epoch 10 => Loss: 0.23927009105682373
Epoch 11 => Loss: 0.23927009105682373
Epoch 12 => Loss: 0.23927009105682373
Epoch 13 => Loss: 0.23927009105682373
Epoch 14 => Loss: 0.23927009105682373
Epoch 15 => Loss: 0.23927009105682373


In [62]:
torch.save(weights, "deep_nnet_weights.pt")

In [63]:
accuracy(weights, test_features)

tensor(0.7607)